## Initialize Libraries and Data

In [6]:
import os
import pandas as pd
from json import loads, dumps
# import numpy as np
# import plotly.express as px
# import plotly.graph_objects as go

In [7]:
# get the path to the data, not in the same location as the jupyter notebook
pathName = os.path.abspath(os.getcwd()) + "\\..\\data\\toronto-police\\"

tps_data = r"Traffic Tickets King Street Jan 2016 - May 2023.csv"

# create dataframes from csv
df_tps_data = pd.read_csv(pathName + tps_data, parse_dates=["Issued DateTime"])

In [8]:
df_tps_data

,OBJECTID,Issued Date,Issued Time,Issued DateTime,Violation Category and Section,Description,Warning,Fine ($),Intersection,Nearest Intersection_Lat_WGS 1984,Nearest Intersection_Long_WGS 1984
0,1,2018-05-03,1:39,2018-05-03 01:39:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection,Warning,110.0,Portland St/ King St. West,43.644396,-79.400297
1,2,2018-05-09,16:43,2018-05-09 16:43:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection,NaN,110.0,University Ave/King St. West,43.647605,-79.385007
2,3,2018-05-14,3:28,2018-05-14 03:28:00,"HTA, 182(2)",Disobey Sign,Warning,110.0,Portland St/ King St. West,43.644396,-79.400297
3,4,2016-07-21,14:24,2016-07-21 14:24:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection,NaN,110.0,Bay St/King St. West,43.648593,-79.380479
4,5,2016-09-30,14:11,2016-09-30 14:11:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection,NaN,110.0,Bay St/King St. West,43.648593,-79.380479
...,...,...,...,...,...,...,...,...,...,...,...
61138,68892,2023-03-18,13:20,2023-03-18 13:20:00,"HTA, 73(3)",Colour Coating Obscuring Interior,NaN,85.0,Bathurst St/ King St. West,43.643869,-79.402905
61139,68893,2023-03-20,17:54,2023-03-20 17:54:00,"HTA, 182(2)",Disobey Sign,Warning,85.0,Spadina Ave/King St. West,43.645413,-79.395247
61140,68894,2023-04-20,19:29,2023-04-20 19:29:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection,Warning,85.0,Victoria St/King St. East,43.649348,-79.376932
61141,68895,2023-05-06,4:15,2023-05-06 04:15:00,"HTA, 3",Vehicle Modified - Fail To Apply For Permit,NaN,85.0,Portland St/ King St. West,43.644396,-79.400297


In [9]:
df_tps_data.dtypes

OBJECTID                                       int64
Issued Date                                   object
Issued Time                                   object
Issued DateTime                       datetime64[ns]
Violation Category and Section                object
Description                                   object
Warning                                       object
Fine ($)                                     float64
Intersection                                  object
Nearest Intersection_Lat_WGS 1984            float64
Nearest Intersection_Long_WGS 1984           float64
dtype: object

## Clean Dataframe

### Clean columns

In [10]:
# Fine column had "Not Available" text removed and are blanks in csv; appear as NaN

# replace NaN in Warning column with "Ticket" since they are actually tickets
df_tps_data["Warning"] = df_tps_data["Warning"].fillna("Ticket")

# rename columns
df_tps_data = df_tps_data.rename(columns={"Violation Category and Section" : "Violation", 
                                          "Warning" : "Issue Type",
                                          "Fine ($)" : "Fine",
                                          "Nearest Intersection_Lat_WGS 1984" : "Nearest Inter Lat",
                                          "Nearest Intersection_Long_WGS 1984" : "Nearest Inter Long"})

# drop date and time columns
df_tps_data = df_tps_data.drop(columns=["Issued Date", "Issued Time"])

In [11]:
# view record with fine = NaN
df_tps_data.loc[df_tps_data["OBJECTID"] == 66]

,OBJECTID,Issued DateTime,Violation,Description,Issue Type,Fine,Intersection,Nearest Inter Lat,Nearest Inter Long
57,66,2019-12-28 00:29:00,"HTA, 53(1)",Driving While Under Suspension,Ticket,NaN,Spadina Ave/King St. West,43.645413,-79.395247


### Clean rows

#### relevant intersections

In [12]:
df_tps_data["Intersection"].unique()

array(['Portland St/ King St. West', 'University Ave/King St. West',
       'Bay St/King St. West', 'John St/King St. West',
       'Spadina Ave/King St. West', 'Victoria St/King St. East',
       'Church St/King St. East', 'Bathurst St/ King St. West',
       'Waterloo Ter/King St. West ', 'Charlotte St/King St. West',
       'Yonge St/King St. West', 'Tecumseth St/ King St. West',
       'Jarvis St/King St East', 'Peter St/King St. West',
       'Brant Pl / King St. West', 'Strachan Ave/ King St. West',
       'York St/King St. West', 'Stafford St/ King St. West',
       'Emily St/King St. West', 'Niagara St/ King St. West',
       'Waterloo Ter/King St. West', 'Brant St/King St. West',
       'Widmer St/King St. West', 'Walnut Ave/ King St. West',
       'Toronto St/King St. East', 'Jordan St/King St. West',
       'Simcoe St/King St. West', 'Sherbourne St/King St. East',
       'Ed Mirvish Way/King St. West', 'George St/King St. East',
       'Frederick St/King St. East'], dtype=ob

In [13]:
intersections_list = [
    'Portland St/ King St. West',
    'University Ave/King St. West',
    'Bay St/King St. West',
    'John St/King St. West',
    'Spadina Ave/King St. West',
    'Church St/King St. East',
    'Bathurst St/ King St. West',
    'Yonge St/King St. West',
    'Jarvis St/King St East',
    'Peter St/King St. West',
    'York St/King St. West',
    'Simcoe St/King St. West'
    ]

In [14]:
# create dataframe with only data of relevant intersections
rel1 = []
for intersection in intersections_list:
    d = df_tps_data[df_tps_data["Intersection"] == intersection]
    rel1.append(d)

df_rel1 = pd.concat(rel1)
df_rel1

,OBJECTID,Issued DateTime,Violation,Description,Issue Type,Fine,Intersection,Nearest Inter Lat,Nearest Inter Long
0,1,2018-05-03 01:39:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection,Warning,110.0,Portland St/ King St. West,43.644396,-79.400297
2,3,2018-05-14 03:28:00,"HTA, 182(2)",Disobey Sign,Warning,110.0,Portland St/ King St. West,43.644396,-79.400297
15,19,2018-01-13 02:18:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection,Ticket,110.0,Portland St/ King St. West,43.644396,-79.400297
20,24,2018-05-17 17:07:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection,Ticket,110.0,Portland St/ King St. West,43.644396,-79.400297
24,28,2018-07-26 16:54:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection,Ticket,110.0,Portland St/ King St. West,43.644396,-79.400297
...,...,...,...,...,...,...,...,...,...
59906,67518,2019-05-24 09:15:00,"HTA, 144(18)",Red Light- Fail To Stop,Ticket,325.0,Simcoe St/King St. West,43.647338,-79.386311
60016,67641,2018-12-19 18:07:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection,Ticket,110.0,Simcoe St/King St. West,43.647338,-79.386311
60595,68283,2016-02-26 13:15:00,"HTA, 33(1)",Driver Fail To Surrender Licence,Ticket,110.0,Simcoe St/King St. West,43.647338,-79.386311
60605,68295,2017-05-16 08:26:00,"HTA, 106(2)",Driver - Fail To Properly Wear Seat Belt,Ticket,200.0,Simcoe St/King St. West,43.647338,-79.386311


#### relevant offences

In [15]:
l = df_rel1["Description"].unique()

for i in sorted(l):
    print(i)

Amber Arrow - Fail To Stop
Amber Light - Fail To Stop
Approach Open Street Car Door Too Closely
Approach Open Street Car Door Too Closely - Community Safety Zone
Arrange For Passenger  Pickup For Compensation  Without Authority
Attempt To Pass - Roadway  Not Clear - Overtaking Traffic
Brake Lights - Improper  Colour
Careless Driving
Careless Driving Causing Bodily Harm Or Death
Change Lane - Not In Safety
Class G1 Licence Holder  Accompanying Driver'S  B.A.C. Above Zero
Class G1 Licence Holder  Carry Front Seat Passenger
Class G1 Licence Holder  Drive At Unlawful Hour
Class G1 Licence Holder  Drive At Unlawful Hour (0000-0500 Hrs)
Class G1 Licence Holder  Unaccompanied By Qualified Driver
Class G2 Licence Holder  (Less Than 6 Months) Excess Passengers Under  20
Class G2 Licence Holder  More Passengers Than Seat Belts
Colour Coating Obscuring Interior
Confuse Identify Of Plate
Contravene Licence Condition - Driving  Ability
Cyclist - Fail To Stop
Cyclist- Ride In Crosswalk
Deface Plate


In [16]:
df_rel1[df_rel1["Description"] == "Use Centre Lane Improperly"]

,OBJECTID,Issued DateTime,Violation,Description,Issue Type,Fine,Intersection,Nearest Inter Lat,Nearest Inter Long
39451,44360,2021-03-24 14:51:00,"HTA, 154(1)(b)",Use Centre Lane Improperly,Warning,110.0,University Ave/King St. West,43.647605,-79.385007


In [17]:
offences_list = [
    "Disobey Sign",
    "Disobey Sign - Community Safety Zone",
    "Disobey Sign At Tunnel",
    "Fail To Obey Lane Sign",
    "Proceed Contrary To  Sign At Intersection",
    "Proceed Contrary To Sign At Intersection -  Community Safety Zone",
    "Use Centre Lane Improperly"
]

In [18]:
# create dataframe with only data of relevant offences
rel2 = []
for offence in offences_list:
    d = df_rel1[df_rel1["Description"] == offence]
    rel2.append(d)

df_rel2 = pd.concat(rel2)
df_rel2

,OBJECTID,Issued DateTime,Violation,Description,Issue Type,Fine,Intersection,Nearest Inter Lat,Nearest Inter Long
2,3,2018-05-14 03:28:00,"HTA, 182(2)",Disobey Sign,Warning,110.0,Portland St/ King St. West,43.644396,-79.400297
25,30,2018-08-15 17:06:00,"HTA, 182(2)",Disobey Sign,Warning,110.0,Portland St/ King St. West,43.644396,-79.400297
40,46,2019-05-16 14:35:00,"HTA, 182(2)",Disobey Sign,Ticket,110.0,Portland St/ King St. West,43.644396,-79.400297
66,76,2020-03-20 16:00:00,"HTA, 182(2)",Disobey Sign,Warning,110.0,Portland St/ King St. West,43.644396,-79.400297
67,77,2020-03-27 12:13:00,"HTA, 182(2)",Disobey Sign,Warning,110.0,Portland St/ King St. West,43.644396,-79.400297
...,...,...,...,...,...,...,...,...,...
57748,65074,2019-12-21 23:47:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection - Co...,Ticket,150.0,Peter St/King St. West,43.645997,-79.392528
59341,66881,2019-12-28 02:48:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection - Co...,Ticket,150.0,Peter St/King St. West,43.645997,-79.392528
60515,68197,2019-12-15 00:10:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection - Co...,Ticket,150.0,Peter St/King St. West,43.645997,-79.392528
32052,35999,2023-03-08 17:35:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection - Co...,Ticket,120.0,York St/King St. West,43.647867,-79.383779


### Temporal Masks

In [19]:
# Taxi Masks
TaxiMask = (df_rel2["Issued DateTime"].dt.hour >= 5) & (df_rel2["Issued DateTime"].dt.hour < 22)
df_tps_nt = df_rel2[TaxiMask].copy()

In [20]:
# Transit Corridor Operating Dates Mask (Nov 12, 2017 onwards)
YearMask = (df_tps_nt["Issued DateTime"] >= '2017-11-12')
df_tps_ntnp = df_tps_nt[YearMask].copy()

In [21]:
df_tps_ntnp = df_tps_ntnp.reset_index(drop=True)

## fun numbers

In [22]:
df_tps_ntnp

,OBJECTID,Issued DateTime,Violation,Description,Issue Type,Fine,Intersection,Nearest Inter Lat,Nearest Inter Long
0,30,2018-08-15 17:06:00,"HTA, 182(2)",Disobey Sign,Warning,110.0,Portland St/ King St. West,43.644396,-79.400297
1,46,2019-05-16 14:35:00,"HTA, 182(2)",Disobey Sign,Ticket,110.0,Portland St/ King St. West,43.644396,-79.400297
2,76,2020-03-20 16:00:00,"HTA, 182(2)",Disobey Sign,Warning,110.0,Portland St/ King St. West,43.644396,-79.400297
3,77,2020-03-27 12:13:00,"HTA, 182(2)",Disobey Sign,Warning,110.0,Portland St/ King St. West,43.644396,-79.400297
4,146,2018-03-16 14:53:00,"HTA, 182(2)",Disobey Sign,Warning,110.0,Portland St/ King St. West,43.644396,-79.400297
...,...,...,...,...,...,...,...,...,...
35307,22307,2019-12-23 16:18:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection - Co...,Ticket,150.0,Peter St/King St. West,43.645997,-79.392528
35308,56820,2019-12-16 15:56:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection - Co...,Ticket,150.0,Peter St/King St. West,43.645997,-79.392528
35309,58675,2019-05-02 16:29:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection - Co...,Ticket,110.0,Peter St/King St. West,43.645997,-79.392528
35310,35999,2023-03-08 17:35:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection - Co...,Ticket,120.0,York St/King St. West,43.647867,-79.383779


In [23]:
df_tps_ntnp[df_tps_ntnp["Description"] == "Disobey Sign At Tunnel"]

,OBJECTID,Issued DateTime,Violation,Description,Issue Type,Fine,Intersection,Nearest Inter Lat,Nearest Inter Long
4297,31493,2018-07-22 14:55:00,"HTA, 183(2)",Disobey Sign At Tunnel,Ticket,110.0,Spadina Ave/King St. West,43.645413,-79.395247


In [24]:
df_tps_ntnp[df_tps_ntnp["Description"] == "Use Centre Lane Improperly"]

,OBJECTID,Issued DateTime,Violation,Description,Issue Type,Fine,Intersection,Nearest Inter Lat,Nearest Inter Long
35311,44360,2021-03-24 14:51:00,"HTA, 154(1)(b)",Use Centre Lane Improperly,Warning,110.0,University Ave/King St. West,43.647605,-79.385007


In [25]:
df_tps_ntnp_vsum = df_tps_ntnp.groupby(["Violation", "Description"])["Fine"].count()
df_tps_ntnp_vsum

Violation                  Description                                                      
HTA, 144(9)                Proceed Contrary To  Sign At Intersection                            30960
                           Proceed Contrary To Sign At Intersection -  Community Safety Zone       43
HTA, 154(1)(b)             Use Centre Lane Improperly                                               1
HTA, 154(1)(c)             Fail To Obey Lane Sign                                                  10
HTA, 182(2)                Disobey Sign                                                          4292
                           Disobey Sign - Community Safety Zone                                     5
HTA, 183(2)                Disobey Sign At Tunnel                                                   1
Name: Fine, dtype: int64

In [26]:
hta182_intersectiongroup = df_tps_ntnp[df_tps_ntnp["Description"] == "Disobey Sign"].groupby("Intersection").count()
hta182_intersectiongroup

,OBJECTID,Issued DateTime,Violation,Description,Issue Type,Fine,Nearest Inter Lat,Nearest Inter Long
Intersection,,,,,,,,
Bathurst St/ King St. West,638,638,638,638,638,638,638,638
Bay St/King St. West,52,52,52,52,52,52,52,52
Church St/King St. East,396,396,396,396,396,396,396,396
Jarvis St/King St East,221,221,221,221,221,221,221,221
John St/King St. West,15,15,15,15,15,15,15,15
Peter St/King St. West,249,249,249,249,249,249,249,249
Portland St/ King St. West,1847,1847,1847,1847,1847,1847,1847,1847
Simcoe St/King St. West,8,8,8,8,8,8,8,8
Spadina Ave/King St. West,263,263,263,263,263,263,263,263


In [27]:
hta144_intersectiongroup = df_tps_ntnp[df_tps_ntnp["Description"] == "Proceed Contrary To  Sign At Intersection"].groupby("Intersection").count()
hta144_intersectiongroup

,OBJECTID,Issued DateTime,Violation,Description,Issue Type,Fine,Nearest Inter Lat,Nearest Inter Long
Intersection,,,,,,,,
Bathurst St/ King St. West,5760,5760,5760,5760,5760,5760,5760,5760
Bay St/King St. West,247,247,247,247,247,247,247,247
Church St/King St. East,1912,1912,1912,1912,1912,1912,1912,1912
Jarvis St/King St East,1116,1116,1116,1116,1116,1116,1116,1116
John St/King St. West,46,46,46,46,46,46,46,46
Peter St/King St. West,3288,3288,3288,3288,3288,3288,3288,3288
Portland St/ King St. West,9577,9577,9577,9577,9577,9577,9577,9577
Simcoe St/King St. West,24,24,24,24,24,24,24,24
Spadina Ave/King St. West,2318,2318,2318,2318,2318,2318,2318,2318


## warnings and tickets for each year/month

In [28]:
df_tps_nt = df_tps_nt.reset_index(drop=True)

In [29]:
df_tps_ntnp["Year"] = df_tps_ntnp["Issued DateTime"].dt.year
df_tps_ntnp["Month"] = df_tps_ntnp["Issued DateTime"].dt.month

df_tps_nt["Year"] = df_tps_nt["Issued DateTime"].dt.year
df_tps_nt["Month"] = df_tps_nt["Issued DateTime"].dt.month

In [38]:
df_tps_nt

,OBJECTID,Issued DateTime,Violation,Description,Issue Type,Fine,Intersection,Nearest Inter Lat,Nearest Inter Long,Year,Month
0,30,2018-08-15 17:06:00,"HTA, 182(2)",Disobey Sign,Warning,110.0,Portland St/ King St. West,43.644396,-79.400297,2018,8
1,46,2019-05-16 14:35:00,"HTA, 182(2)",Disobey Sign,Ticket,110.0,Portland St/ King St. West,43.644396,-79.400297,2019,5
2,76,2020-03-20 16:00:00,"HTA, 182(2)",Disobey Sign,Warning,110.0,Portland St/ King St. West,43.644396,-79.400297,2020,3
3,77,2020-03-27 12:13:00,"HTA, 182(2)",Disobey Sign,Warning,110.0,Portland St/ King St. West,43.644396,-79.400297,2020,3
4,146,2018-03-16 14:53:00,"HTA, 182(2)",Disobey Sign,Warning,110.0,Portland St/ King St. West,43.644396,-79.400297,2018,3
...,...,...,...,...,...,...,...,...,...,...,...
36852,22307,2019-12-23 16:18:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection - Co...,Ticket,150.0,Peter St/King St. West,43.645997,-79.392528,2019,12
36853,56820,2019-12-16 15:56:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection - Co...,Ticket,150.0,Peter St/King St. West,43.645997,-79.392528,2019,12
36854,58675,2019-05-02 16:29:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection - Co...,Ticket,110.0,Peter St/King St. West,43.645997,-79.392528,2019,5
36855,35999,2023-03-08 17:35:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection - Co...,Ticket,120.0,York St/King St. West,43.647867,-79.383779,2023,3


In [30]:
df_tps_ntnp_msum = df_tps_ntnp.groupby(["Year", "Month", "Issue Type"]).size().reset_index(name='count')
df_tps_nt_msum = df_tps_nt.groupby(["Year", "Month", "Issue Type"]).size().reset_index(name='count')
df_tps_ntnp_msum

,Year,Month,Issue Type,count
0,2017,11,Ticket,677
1,2017,11,Warning,1091
2,2017,12,Ticket,536
3,2017,12,Warning,167
4,2018,1,Ticket,507
...,...,...,...,...
129,2023,3,Warning,156
130,2023,4,Ticket,365
131,2023,4,Warning,122
132,2023,5,Ticket,466


#### getting properly formatted json

In [47]:
df_tickets = df_tps_nt[df_tps_nt["Issue Type"] == "Ticket"].groupby(["Year", "Month", "Issue Type"]).size().reset_index(name="Tickets")
df_tickets = df_tickets.drop(columns=["Issue Type"])
df_tickets

,Year,Month,Tickets
0,2016,1,39
1,2016,2,41
2,2016,3,77
3,2016,4,73
4,2016,5,77
...,...,...,...
84,2023,1,461
85,2023,2,538
86,2023,3,390
87,2023,4,365


In [48]:
df_warnings = df_tps_nt[df_tps_nt["Issue Type"] == "Warning"].groupby(["Year", "Month", "Issue Type"]).size().reset_index(name="Warnings")
df_warnings = df_warnings.drop(columns=["Issue Type"])
df_warnings

,Year,Month,Warnings
0,2016,1,12
1,2016,2,12
2,2016,3,8
3,2016,4,9
4,2016,5,15
...,...,...,...
84,2023,1,135
85,2023,2,172
86,2023,3,156
87,2023,4,122


In [50]:
df = df_tickets.merge(df_warnings, left_on=["Year", "Month"], right_on=["Year", "Month"])
df

,Year,Month,Tickets,Warnings
0,2016,1,39,12
1,2016,2,41,12
2,2016,3,77,8
3,2016,4,73,9
4,2016,5,77,15
...,...,...,...,...
84,2023,1,461,135
85,2023,2,538,172
86,2023,3,390,156
87,2023,4,365,122


In [51]:
result = df.to_json(orient="records")
parsed = loads(result)
dumps(parsed)

'[{"Year": 2016, "Month": 1, "Tickets": 39, "Warnings": 12}, {"Year": 2016, "Month": 2, "Tickets": 41, "Warnings": 12}, {"Year": 2016, "Month": 3, "Tickets": 77, "Warnings": 8}, {"Year": 2016, "Month": 4, "Tickets": 73, "Warnings": 9}, {"Year": 2016, "Month": 5, "Tickets": 77, "Warnings": 15}, {"Year": 2016, "Month": 6, "Tickets": 102, "Warnings": 12}, {"Year": 2016, "Month": 7, "Tickets": 60, "Warnings": 11}, {"Year": 2016, "Month": 8, "Tickets": 29, "Warnings": 7}, {"Year": 2016, "Month": 9, "Tickets": 62, "Warnings": 5}, {"Year": 2016, "Month": 10, "Tickets": 72, "Warnings": 6}, {"Year": 2016, "Month": 11, "Tickets": 71, "Warnings": 5}, {"Year": 2016, "Month": 12, "Tickets": 62, "Warnings": 4}, {"Year": 2017, "Month": 1, "Tickets": 105, "Warnings": 9}, {"Year": 2017, "Month": 2, "Tickets": 43, "Warnings": 13}, {"Year": 2017, "Month": 3, "Tickets": 88, "Warnings": 6}, {"Year": 2017, "Month": 4, "Tickets": 48, "Warnings": 5}, {"Year": 2017, "Month": 5, "Tickets": 79, "Warnings": 4}, {

## tickets/warnings for each intersection for 2019

^ total for the year, simple table

In [63]:
df_tps_nt2019 = df_tps_nt[df_tps_nt["Year"] == 2019]

In [64]:
df_tps_nt2019.groupby(["Intersection"]).size().reset_index(name="count")

,Intersection,count
0,Bathurst St/ King St. West,1019
1,Bay St/King St. West,30
2,Church St/King St. East,565
3,Jarvis St/King St East,259
4,John St/King St. West,14
5,Peter St/King St. West,969
6,Portland St/ King St. West,2218
7,Simcoe St/King St. West,8
8,Spadina Ave/King St. West,677
9,University Ave/King St. West,791


## sanity check with news article ticket numbers

In [68]:
SanityMask = (df_tps_nt["Issued DateTime"] >= '2017-11-12') & (df_tps_nt["Issued DateTime"] < '2019-08-01')
df_sanity = df_tps_nt[SanityMask].copy()
df_sanity

# 16,000 tickets for the Corridor from November 2017 to July 2019

,OBJECTID,Issued DateTime,Violation,Description,Issue Type,Fine,Intersection,Nearest Inter Lat,Nearest Inter Long,Year,Month
0,30,2018-08-15 17:06:00,"HTA, 182(2)",Disobey Sign,Warning,110.0,Portland St/ King St. West,43.644396,-79.400297,2018,8
1,46,2019-05-16 14:35:00,"HTA, 182(2)",Disobey Sign,Ticket,110.0,Portland St/ King St. West,43.644396,-79.400297,2019,5
4,146,2018-03-16 14:53:00,"HTA, 182(2)",Disobey Sign,Warning,110.0,Portland St/ King St. West,43.644396,-79.400297,2018,3
5,148,2018-02-16 19:07:00,"HTA, 182(2)",Disobey Sign,Warning,110.0,Portland St/ King St. West,43.644396,-79.400297,2018,2
6,172,2018-05-19 12:33:00,"HTA, 182(2)",Disobey Sign,Warning,110.0,Portland St/ King St. West,43.644396,-79.400297,2018,5
...,...,...,...,...,...,...,...,...,...,...,...
36827,14852,2017-11-17 07:34:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection - Co...,Warning,150.0,Spadina Ave/King St. West,43.645413,-79.395247,2017,11
36834,57445,2019-03-16 18:05:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection - Co...,Warning,150.0,Church St/King St. East,43.649865,-79.374613,2019,3
36841,9825,2019-01-30 10:28:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection - Co...,Ticket,150.0,Yonge St/King St. West,43.649091,-79.378118,2019,1
36842,15267,2018-06-22 21:59:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection - Co...,Warning,150.0,Yonge St/King St. West,43.649091,-79.378118,2018,6


In [67]:
NewsMask = (df_tps_nt["Issued DateTime"] >= '2023-01-01') & (df_tps_nt["Issued DateTime"] < '2023-03-18')
df_news = df_tps_nt[NewsMask].copy()
df_news

# From January to March 17, 2023, 1852 tickets were issued (incl warnings)

,OBJECTID,Issued DateTime,Violation,Description,Issue Type,Fine,Intersection,Nearest Inter Lat,Nearest Inter Long,Year,Month
172,6402,2023-01-19 09:57:00,"HTA, 182(2)",Disobey Sign,Ticket,85.0,Portland St/ King St. West,43.644396,-79.400297,2023,1
260,10011,2023-02-05 12:35:00,"HTA, 182(2)",Disobey Sign,Ticket,85.0,Portland St/ King St. West,43.644396,-79.400297,2023,2
314,12037,2023-01-19 10:56:00,"HTA, 182(2)",Disobey Sign,Ticket,85.0,Portland St/ King St. West,43.644396,-79.400297,2023,1
321,12335,2023-01-06 07:54:00,"HTA, 182(2)",Disobey Sign,Ticket,85.0,Portland St/ King St. West,43.644396,-79.400297,2023,1
366,13883,2023-03-10 09:06:00,"HTA, 182(2)",Disobey Sign,Warning,85.0,Portland St/ King St. West,43.644396,-79.400297,2023,3
...,...,...,...,...,...,...,...,...,...,...,...
36843,19519,2023-03-11 18:15:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection - Co...,Ticket,120.0,Yonge St/King St. West,43.649091,-79.378118,2023,3
36844,43973,2023-02-26 14:10:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection - Co...,Warning,120.0,Yonge St/King St. West,43.649091,-79.378118,2023,2
36847,26980,2023-03-12 07:42:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection - Co...,Warning,120.0,Jarvis St/King St East,43.650407,-79.372114,2023,3
36848,1305,2023-03-13 12:46:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection - Co...,Warning,120.0,Peter St/King St. West,43.645997,-79.392528,2023,3


In [77]:
CityNewsMask = (df_tps_nt["Issued DateTime"] >= '2019-04-01') & (df_tps_nt["Issued DateTime"] < '2023-09-01')
df_citynews = df_tps_nt[CityNewsMask].copy()
df_citynews

,OBJECTID,Issued DateTime,Violation,Description,Issue Type,Fine,Intersection,Nearest Inter Lat,Nearest Inter Long,Year,Month
1,46,2019-05-16 14:35:00,"HTA, 182(2)",Disobey Sign,Ticket,110.0,Portland St/ King St. West,43.644396,-79.400297,2019,5
2,76,2020-03-20 16:00:00,"HTA, 182(2)",Disobey Sign,Warning,110.0,Portland St/ King St. West,43.644396,-79.400297,2020,3
3,77,2020-03-27 12:13:00,"HTA, 182(2)",Disobey Sign,Warning,110.0,Portland St/ King St. West,43.644396,-79.400297,2020,3
8,195,2019-05-14 15:50:00,"HTA, 182(2)",Disobey Sign,Ticket,110.0,Portland St/ King St. West,43.644396,-79.400297,2019,5
9,217,2020-02-13 16:53:00,"HTA, 182(2)",Disobey Sign,Ticket,110.0,Portland St/ King St. West,43.644396,-79.400297,2020,2
...,...,...,...,...,...,...,...,...,...,...,...
36852,22307,2019-12-23 16:18:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection - Co...,Ticket,150.0,Peter St/King St. West,43.645997,-79.392528,2019,12
36853,56820,2019-12-16 15:56:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection - Co...,Ticket,150.0,Peter St/King St. West,43.645997,-79.392528,2019,12
36854,58675,2019-05-02 16:29:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection - Co...,Ticket,110.0,Peter St/King St. West,43.645997,-79.392528,2019,5
36855,35999,2023-03-08 17:35:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection - Co...,Ticket,120.0,York St/King St. West,43.647867,-79.383779,2023,3


In [81]:
# df_citynews[df_citynews["Issue Type"] == "Ticket"].groupby(["Year"]).size().reset_index(name="count")
df_citynews.groupby(["Year"]).size().reset_index(name="count")

,Year,count
0,2019,4733
1,2020,4337
2,2021,3173
3,2022,3118
4,2023,2220


In [72]:
df_tps_ntnp.groupby(["Year"]).size().reset_index(name="count")

,Year,count
0,2017,2471
1,2018,7982
2,2019,7516
3,2020,6089
4,2021,4427
5,2022,3904
6,2023,2923


In [80]:
df_tps_nt.sort_values(by=["Issued DateTime"])

,OBJECTID,Issued DateTime,Violation,Description,Issue Type,Fine,Intersection,Nearest Inter Lat,Nearest Inter Long,Year,Month
19292,53377,2016-01-05 18:14:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection,Ticket,110.0,Bay St/King St. West,43.648593,-79.380479,2016,1
18311,11808,2016-01-05 18:32:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection,Ticket,110.0,Bay St/King St. West,43.648593,-79.380479,2016,1
18485,19161,2016-01-07 10:16:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection,Ticket,110.0,Bay St/King St. West,43.648593,-79.380479,2016,1
1977,30107,2016-01-07 15:29:00,"HTA, 182(2)",Disobey Sign,Warning,110.0,University Ave/King St. West,43.647605,-79.385007,2016,1
3336,39091,2016-01-08 08:59:00,"HTA, 182(2)",Disobey Sign,Ticket,110.0,Bathurst St/ King St. West,43.643869,-79.402905,2016,1
...,...,...,...,...,...,...,...,...,...,...,...
29116,61739,2023-05-31 20:27:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection,Ticket,85.0,Bathurst St/ King St. West,43.643869,-79.402905,2023,5
28478,53609,2023-05-31 20:44:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection,Ticket,85.0,Bathurst St/ King St. West,43.643869,-79.402905,2023,5
33983,12635,2023-05-31 21:07:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection,Ticket,85.0,Peter St/King St. West,43.645997,-79.392528,2023,5
20434,21164,2023-05-31 21:21:00,"HTA, 144(9)",Proceed Contrary To Sign At Intersection,Ticket,85.0,Spadina Ave/King St. West,43.645413,-79.395247,2023,5
